In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import TABLEAU_COLORS

# Read and concatenate all CSVs
file_paths = [
    "combined_results/anl_asymmetric.csv",
    "combined_results/anl_symmetric.csv",
    "combined_results/students_asymmetric.csv",
    "combined_results/students_symmetric.csv",
    "combined_results/baseline_asymmetric.csv",
    "combined_results/baseline_symmetric.csv",
    "combined_results/our_asymmetric.csv",
    "combined_results/our_symmetric.csv",
]

# Load all dataframes
dfs = {}
for path in file_paths:
    key = path.split('/')[-1].split('.')[0]  # e.g., 'anl_asymmetric'
    dfs[key] = pd.read_csv(path, index_col=0)

# Get list of our agents, sort numerically, and simplify names
our_agents = dfs['our_asymmetric'].index.tolist()
our_agents_sorted = sorted(our_agents, key=lambda x: int(x[-2:]))  # Sort by last two digits
simplified_names = [f"Agent {int(agent[-2:])}" for agent in our_agents_sorted]  # Convert to Agent 01 format

# Metrics to plot
metrics = ['avg_utility', 'avg_nash_product', 'avg_social_welfare', 'avg_num_offers']
metric_names = {
    'avg_utility': 'Average Utility',
    'avg_nash_product': 'Average Nash Product',
    'avg_social_welfare': 'Average Social Welfare',
    'avg_num_offers': 'Average Number of Offers'
}

# Opponent categories and their corresponding data keys
opponent_categories = {
    'Students': ['students_asymmetric', 'students_symmetric'],
    'Baseline': ['baseline_asymmetric', 'baseline_symmetric'],
    'ANL2022': ['anl_asymmetric', 'anl_symmetric'],
    'Our Agents': ['our_asymmetric', 'our_symmetric']
}

# Distinct colors for each opponent category
colors = list(TABLEAU_COLORS.values())[:4]  # First 4 Tableau colors

# Create plots for each metric
for metric in metrics:
    plt.figure(figsize=(14, 8))
    
    # For each agent, collect performance against each opponent type
    agent_data = {}
    for agent in our_agents_sorted:
        agent_data[agent] = []
        for category, keys in opponent_categories.items():
            # Get data from both domains
            values = []
            for key in keys:
                if agent in dfs[key].index:
                    values.append(dfs[key].loc[agent, metric])
            # Average across domains
            avg_value = sum(values) / len(values) if values else 0
            agent_data[agent].append(avg_value)
    
    # Plot settings
    n_agents = len(our_agents_sorted)
    n_categories = len(opponent_categories)
    bar_width = 0.2
    index = np.arange(n_agents)
    
    # Create bars for each opponent category
    for i, (category, _) in enumerate(opponent_categories.items()):
        values = [agent_data[agent][i] for agent in our_agents_sorted]
        plt.bar(index + i * bar_width, values, bar_width, 
                label=category, color=colors[i])
    
    # Formatting
    plt.title(f'{metric_names[metric]} by Agent and Opponent Type', pad=20)
    plt.ylabel(metric_names[metric])
    
    # Use simplified agent names and remove x-axis label
    plt.xticks(index + bar_width * (n_categories - 1) / 2, 
               simplified_names, rotation=45)
    plt.xlabel('')  # Remove x-axis label
    
    # Legend in top-left
    plt.legend(loc='upper left', bbox_to_anchor=(0, 1))
    
    # Set y-axis range for social welfare
    if metric == 'avg_social_welfare':
        plt.ylim(0, 1.65)
    
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    
    # Save plot
    plt.savefig(f'{metric}_by_agent_and_opponent.png', dpi=300, bbox_inches='tight')
    plt.close()

print("Plots saved as:")
for metric in metrics:
    print(f"- {metric}_by_agent_and_opponent.png")

Plots saved as:
- avg_utility_by_agent_and_opponent.png
- avg_nash_product_by_agent_and_opponent.png
- avg_social_welfare_by_agent_and_opponent.png
- avg_num_offers_by_agent_and_opponent.png
